In [ ]:
#run
!pip install --quiet langchain langchain-anthropic langchain-community faiss-cpu pypdf python-dotenv langchain_mistralai langchain_deepseek langchain_cohere asyncio psutil GPUtil supabase

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
#run
#Importing the packages
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from math import log
from collections import Counter
import itertools
import re
import supabase
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import SupabaseVectorStore
from langchain_openai import OpenAIEmbeddings  # New package as per deprecation warning
from langchain.vectorstores import SupabaseVectorStore
from langchain.chat_models import ChatOpenAI  # For GPT-4 and as a placeholder for Mistral
from langchain.chat_models import ChatAnthropic  # For Claude (ensure compatibility with your LangChain version)
from langchain.schema import SystemMessage, HumanMessage
from langchain_anthropic import ChatAnthropic
import re
from datetime import datetime
from supabase import create_client, Client
from langchain_mistralai import ChatMistralAI
from langchain_cohere import ChatCohere
from langchain_deepseek import ChatDeepSeek
# from langchain_google_genai import ChatGoogleGenerativeAI



In [ ]:
# -------------------------------
# 1. Read CSV Data
# -------------------------------
from google.colab import drive # VH
drive.mount('/content/drive') # VH
file_path = "/content/drive/My Drive/Internship DTSC/Dataset Length Bias 0407.csv" # VH
df = pd.read_csv(file_path)
print("CSV loaded with {} records.".format(len(df)))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CSV loaded with 147 records.


In [ ]:
##run
#REading the API key
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OpenAI')
supabase_key = userdata.get('Supabase_key') #vector store
MISTRAL_API_KEY = userdata.get('Mistral')
CLAUDE_API_KEY = userdata.get('Anthropic')
COHERE_API_KEY = userdata.get('Cohere')
DEEPSEEK_API_KEY = userdata.get('Deepseek')
# GEMINI_API_KEY = userdata.get('Gemini')


In [ ]:

# -------------------------------
# 2. Connect to Supabase and Create a Vector Store
# -------------------------------
# Note: Install the required supabase and langchain libraries.
# Replace with your actual credentials.

SUPABASE_URL = ""
SUPABASE_KEY = supabase_key
# OPENAI_API_KEY = openai_key # VH delete?

supabase_client = supabase.create_client(SUPABASE_URL, SUPABASE_KEY)
# Initialize your embeddings using the new import:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
# Define the table name where your documents will be stored.
table_name = "retrieval_length"
# Create the Supabase vector store by providing the client, embeddings, and table_name.
vector_store = SupabaseVectorStore(client=supabase_client, embedding=embeddings, table_name=table_name)

In [ ]:
def generate_embedding(text):
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY,model="text-embedding-3-large")  # Use the appropriate model
    response = embeddings.embed_query(text)  # Correctly call the method to generate embeddings
    embedding = response
    return embedding

In [ ]:
# VH Gives error because of unexpected float values in content
# Generate embeddings for each row
df['embedding'] = df['content'].apply(generate_embedding) # do not run too much!

In [ ]:
df.head()

In [ ]:
def insert_documents(df: pd.DataFrame):
    for index, row in df.iterrows():
        print(f"Inserting document with ID: {int(row['id'])}") # VH Previously: print(f"Inserting document with ID: {int(row['id']*10)}")
        data = {
            "id": int(row["id"]*10), # VH Previously: "id": int(row["id"]*10),
            "content": row["content"],
            "metadata": row.get("metadata", None),
            "embedding": row["embedding"]
        }
        response = supabase_client.table("retrieval_length").upsert(data).execute() # VH table name should correspond exactly with given name in supabase / upsert instead of insert



In [ ]:
insert_documents(df)

In [ ]:
def get_embedding(text):
    """Get embeddings for a text using OpenAI's embedding model"""
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY,model="text-embedding-3-large")  # Use the appropriate model
    response = embeddings.embed_query(text)  # Correctly call the method to generate embeddings
    embedding = response
    return embedding

In [ ]:
#Connect to Supabase
 ### Supabase Credentials
SUPABASE_URL = ""
SUPABASE_KEY = supabase_key


# Create Supabase client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

In [ ]:
# -------------------------------
# 3. Define Retrieval Function for Multiple Models with Re-Ranking
# -------------------------------

def retrieve(query, model_name, k=10, re_rank=False):
    """
    Retrieve top-k documents for a query using Supabase vector search with optional LLM re-ranking.

    Parameters:
      query (str): The search query.
      model_name (str): One of 'gpt-4o-mini', 'claude', 'mistral', etc.
      k (int): Number of top documents to retrieve.
      re_rank (bool): Whether to re-rank the documents using the LLM.

    Returns:
      List[dict]: A list of dictionaries with document 'id', 'rank', and 'content'.
    """

    # Initialize LLM
    model_name = model_name.lower()
    if model_name == "gpt":
        llm = ChatOpenAI(model_name="gpt-4o-2024-11-20", openai_api_key=OPENAI_API_KEY)
    elif model_name == "claude":
        llm = ChatAnthropic(model="claude-3-7-sonnet-latest", anthropic_api_key=CLAUDE_API_KEY)
    elif model_name == "mistral":
        llm = ChatMistralAI(model="mistral-large-latest", mistral_api_key=MISTRAL_API_KEY)
    elif model_name == "cohere":
        llm = ChatCohere(model="command-a-03-2025", cohere_api_key=COHERE_API_KEY)
    elif model_name == "deepseek":
        import os
        os.environ["DEEPSEEK_API_KEY"] = DEEPSEEK_API_KEY
        llm = ChatDeepSeek(model="deepseek-v3-chat")
    else:
        raise ValueError(f"Unsupported model: {model_name}")

    try:
        # Step 1: Get embedding
        query_embedding = get_embedding(query)

        # Step 2: Retrieve documents from Supabase via RPC
        response = supabase.rpc(
            'match_documents_length_no_filter',
            {
                'query_embedding': query_embedding,
                'match_count': k
            }
        ).execute()

        if not response.data or len(response.data) == 0:
            print("No relevant documents found.")
            return []

        # Step 3: Build document list
        docs = [
            type("Doc", (object,), {
                "id": doc.get("id"),
                "page_content": doc.get("content", ""),
                "metadata": doc.get("metadata", {})
            })()
            for doc in response.data
        ]

        actual_k = min(k, len(docs))

        # Step 4: Optional re-ranking
        if re_rank and actual_k > 1:
            try:
                documents_text = "\n\n".join([
                    f"Document {i+1} (ID: {docs[i].id}):\n{docs[i].page_content}"
                    for i in range(actual_k)
                ])

                prompt = f"""
                Query: {query}

                You will be given {actual_k} documents retrieved via semantic search.
                Your task is to re-rank these documents in order of their relevance to the query.
                Please return EXACTLY {actual_k} document numbers in order, from MOST to LEAST relevant,
                separated by commas (e.g., "3,1,2").

                Documents:
                {documents_text}
                """

                messages = [
                    SystemMessage(content="You are a helpful assistant skilled at ranking document relevance."),
                    HumanMessage(content=prompt)
                ]

                llm_response = llm.invoke(messages)
                ranking_text = llm_response.content.strip()
                ranking_order = [int(num.strip()) - 1 for num in re.findall(r'\d+', ranking_text)]

                if len(ranking_order) != actual_k or sorted(ranking_order) != list(range(actual_k)):
                    print(f"Invalid ranking received: {ranking_text}. Using default order.")
                    ranking_order = list(range(actual_k))

                docs = [docs[i] for i in ranking_order]

            except Exception as e:
                print(f"Re-ranking failed: {e}. Using initial ranking.")

        # Step 5: Return formatted result
        results = [
            {
                "id": doc.id,
                "rank": idx + 1,
                "content": doc.page_content
            }
            for idx, doc in enumerate(docs)
        ]
        return results

    except Exception as e:
        print(f"Error retrieving documents: {e}")
        return []

In [ ]:
# VH For display in next cell
from IPython.display import display, HTML

In [ ]:
# Example usage (uncomment to test):
results_example = retrieve("How do I connect to Outlook?", "gpt", k=3, re_rank=True)
print(results_example)
#display(HTML(f"<pre style='white-space:pre-wrap;'>{results_example}</pre>")) # VH

<ipython-input-7-e80e3714b995>:22: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-2024-11-20", openai_api_key=OPENAI_API_KEY)


[{'id': 398620, 'rank': 1, 'content': 'output.id: 368d3765-a38d-4f63-bffe-0f59d86be5dc | output.metadata: webUrl":"https://dtservicesandconsulting.sharepoint.com/sites/DTServicesandConsultingSRL/SitePages/Associer-ton-calendrier-Officient-%C3%A0-Outlook-(cong%C3%A9s-des-coll%C3%A8gues).aspx, who created it: "displayName":"Samuel  Aulotte","email":"samuel.aulotte@dtsc.be" and when it was created: "createdDateTime":"2025-02-25T13:27:58Z" and last time it was modified: "lastModifiedDateTime":"2025-02-28T10:00:35Z" | output.content:\n\n1. Access Calendar Settings\n- Open your calendar in Officient.\n- Click on the gear icon to access the settings.\n\n2. Link Your Personal Calendar\n- Select the option "Link your personal calendar."\n- A list of links will be provided for different calendar integrations.\n- Find and copy the link corresponding to your personal calendar.\n- Make sure to keep this link private as it contains access to your calendar data.\n\n3. Add Your Calendar to Outlook\n- 

In [ ]:
# -------------------------------
# 4. Prepare 50 Test Queries        VH: it's 49 now
# -------------------------------
test_queries = [
    "Boîte aux lettres?", # 3985
    "How can I access my Officient Calendar?", # 3986
    "Bannière?", # 3987
    "What are the cafeteria plan benefits?", # 3989
    "What about my car configuration offer?", # 3990
    "How do I create a Canva?", # 3991
    "What about chargemap business?", # 3992
    "What is the Moodboard?", # 3993
    "What about Chargemap (domicile)?", # 3994
    "What about Connecting Expertise?", # 3995
    "What's the BeCentral address?", # 3996
    "What about a Microsoft 365 license?", # 3997
    "What about Google Calendar", # 3998
    "What is Call to Action?", # 3999
    "What are compensatory rest days?", # 4000
    "How do I access the shared library?", # 4001
    "What is the login for StaffIT?", # 4003
    "How to Backup Odoo Contacts?", # 4005
    "How can I export leads from Odoo?", # 4006
    "What is Lean six sigma?", # 4007
    "What is the fleet policy for traffic fines?", # 4008
    "What about dtsc.be performance?", # 4010
    "What is a collaborative mailbox?", # 4011
    "What about a green card?", # 4014
    "Where about Securex?", # 4015
    "How to create a teams meeting from Google Agenda?", # 4017
    "What about Supplementary Family Allowances?", # 4018
    "Core rules Posts?", # 4019
    "What activities are included in the DTeam Spirit Challenge?", # 4020
    "What are the limits for the mobility budget?", # 4021
    "What about Nexxtmove?", # 4024
    "Enable Mail Plugin?", # 4025
    "What about Officient employee self-service?", # 4026
    "What about ECO vouchers?", # 4027
    "What about birth leave?", # 4028
    "What about dtsc.odoo.com?", # 4030
    "What about ProUnity?", # 4031
    "What about a hiring bonus?", # 4032
    "What about Powerdale?", # 4034
    "What about Single Permits?", # 4036
    "What about dbudin@dtsc.be.tapfin?", # 4037
    "What about Elia Recent Job Requisitions?", # 4038
    "What about Subsidies?", # 4040
    "Who are our suppliers?", # 4041
    "What is TED?", # 4042
    "How to activate Music Streaming?", # 4043
    "What are PYXIS and Scrum?", # 4046
    "How to open another mailbox?", # 4047
    "What about BNP Paribas warrants?" # 4048
]

In [ ]:
# VH retrieval test
def process_multiple_questions(questions_list, model="gpt-4o-mini", k=3, re_rank=True, search_method="keyword"):
    """
    Process multiple questions through the retrieve function and display results

    Args:
        questions_list: List of question strings
        model: Model to use for retrieval
        k: Number of results to return
        re_rank: Whether to re-rank results
        search_method: Method for search ("keyword" or other options)
    """
    from IPython.display import HTML, display

    for question in questions_list:
        print(f"\nProcessing question: {question}")
        results = retrieve(question, model, k=k, re_rank=re_rank, search_method=search_method)

        # Handle results based on its type
        if isinstance(results, list):
            # If results is a list, join the items with double newlines
            formatted_results = "\n\n".join([str(item) for item in results])
        else:
            # If results is a string, add line breaks
            if k > 1:
                formatted_results = results.replace("\n", "\n\n")
            else:
                formatted_results = results

        display(HTML(f"<pre style='white-space:pre-wrap;'>{formatted_results}</pre>"))
        print("-" * 80)  # Separator between questions

# Example usage with your questions list:
questions = [
    "Boîte aux lettres?", # 3985
    "How can I access my Officient Calendar?", # 3986
    "Bannière?", # 3987
    "What are the cafeteria plan benefits?", # 3989
    "What about my car configuration offer?", # 3990
    "How do I create a Canva?", # 3991
    "What about chargemap business?", # 3992
    "What is the Moodboard?", # 3993
    "What about Chargemap (domicile)?", # 3994
    "What about Connecting Expertise?", # 3995
    "What's the BeCentral address?", # 3996
    "What about a Microsoft 365 license?", # 3997
    "What about Google Calendar", # 3998
    "What is Call to Action?", # 3999
    "What are compensatory rest days?", # 4000
    "How do I access the shared library?", # 4001
    "What is the login for StaffIT?", # 4003
    "How to Backup Odoo Contacts?", # 4005
    "How can I export leads from Odoo?", # 4006
    "What is Lean six sigma?", # 4007
    "What is the fleet policy for traffic fines?", # 4008
    "What about dtsc.be performance?", # 4010
    "What is a collaborative mailbox?", # 4011
    "What about a green card?", # 4014
    "Where about Securex?", # 4015
    "How to create a teams meeting from Google Agenda?", # 4017
    "What about Supplementary Family Allowances?", # 4018
    "Core rules Posts?", # 4019
    "What activities are included in the DTeam Spirit Challenge?", # 4020
    "What are the limits for the mobility budget?", # 4021
    "What about Nexxtmove?", # 4024
    "Enable Mail Plugin?", # 4025
    "What about Officient employee self-service?", # 4026
    "What about ECO vouchers?", # 4027
    "What about birth leave?", # 4028
    "What about dtsc.odoo.com?", # 4030
    "What about ProUnity?", # 4031
    "What about a hiring bonus?", # 4032
    "What about Powerdale?", # 4034
    "What about Single Permits?", # 4036
    "What about dbudin@dtsc.be.tapfin?", # 4037
    "What about Elia Recent Job Requisitions?", # 4038
    "What about Subsidies?", # 4040
    "Who are our suppliers?", # 4041
    "What is TED?", # 4042
    "How to activate Music Streaming?", # 4043
    "What are PYXIS and Scrum?", # 4046
    "How to open another mailbox?", # 4047
    "What about BNP Paribas warrants?" # 4048
]

# Uncomment to run:
# process_multiple_questions(questions, "gpt-4o-mini", k=3, re_rank=True, search_method="keyword")

In [ ]:
# Ensure exactly 50 queries.    VH: it's 49
if len(test_queries) < 49:
    test_queries = test_queries * (49 // len(test_queries) + 1)
test_queries = test_queries[:49]
print("Prepared {} test queries.".format(len(test_queries)))

Prepared 49 test queries.


In [ ]:
test_queries

In [ ]:
# -------------------------------
# 5. Retrieve Documents for Each Query Across All Models
# -------------------------------
# Initialize a list to collect data

models = ["gpt-4o-mini",  "claude", "mistral", "cohere", "deepseek"]
# retrieval_results structure: { model_name: { query: [list of document results] } }
retrieval_results = {model: {} for model in models}

for model in models:
    for query in test_queries:
        # Set re_rank=True if you wish to re-rank documents using the LLM.
        retrieval_results[model][query] = retrieve(query, model, k=3, re_rank=True, search_method="keyword")
print("Retrieval complete for all models and queries.")

In [ ]:
def analyze_content_word_count(df):
    """
    Analyzes content column and counts words after "output.content: "

    Parameters:
    df (pandas.DataFrame): DataFrame with a 'content' column

    Returns:
    pandas.DataFrame: DataFrame with 'word_count' column added
    """
    # Create a copy of the dataframe
    result_df = df.copy()

    # Count words after marker
    def count_words(text):
        if not isinstance(text, str):
            return 0

        marker = "output.content: "
        pos = text.find(marker)

        if pos == -1:
            return len(text.split())  # Count all words if marker not found

        # Extract content after marker and count words
        content_after_marker = text[pos + len(marker):]
        return len(content_after_marker.split())

    # Apply word counting
    result_df['word_count'] = result_df['content'].apply(count_words)

    return result_df

In [ ]:
# VH Apply word count function and create df

# Initialize a list to collect data
data = []

# Iterate over each model and its corresponding queries
for model, queries in retrieval_results.items():
    for query, documents in queries.items():
        for doc in documents:
            data.append((model, query, doc['rank'], doc['id'], doc['content']))

# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=['Model', 'Query', 'Rank', 'Document ID', 'content'])

# Apply the analyze_content_word_count function to add word count and categories
df = analyze_content_word_count(df)
df = df.drop(columns=['content'])

In [ ]:
df

,Model,Query,Rank,Document ID,word_count
0,gpt-4o-mini,Boîte aux lettres?,1,398420,706
1,gpt-4o-mini,Boîte aux lettres?,2,398400,142
2,gpt-4o-mini,Boîte aux lettres?,3,398410,104
3,gpt-4o-mini,How can I access my Officient Calendar?,1,398600,116
4,gpt-4o-mini,How can I access my Officient Calendar?,2,398620,256
...,...,...,...,...,...
725,deepseek,How to open another mailbox?,2,404710,196
726,deepseek,How to open another mailbox?,3,404720,649
727,deepseek,What about BNP Paribas warrants?,1,404800,46
728,deepseek,What about BNP Paribas warrants?,2,404810,168


In [ ]:
def categorize_word_count(df):
    """
    Categorizes an existing word_count column as 'short', 'medium', or 'long'
    within each Model and Query group based on relative length.

    Parameters:
    df (pandas.DataFrame): DataFrame with 'word_count', 'Model', and 'Query' columns

    Returns:
    pandas.DataFrame: DataFrame with 'length_category' column added
    """
    result_df = df.copy()

    # Define the length categories
    length_categories = ['short', 'medium', 'long']

    # Sort and assign length categories within each group
    result_df['length_category'] = (
        result_df.sort_values(by='word_count', ascending=True)
        .groupby(['Model', 'Query'])
        .cumcount()
        .apply(lambda x: length_categories[min(x, len(length_categories)-1)])
    )

    return result_df

In [ ]:
df = categorize_word_count(df)

In [ ]:
# Set up the plot grid: 2x2 for ranks 1 to 4
fig, axes = plt.subplots(3, 1, figsize=(8, 15))
#fig.suptitle("Length Category Distribution by Model for Each Rank", fontsize=16)

# Flatten axes array for easy indexing#
axes = axes.flatten()

# Loop through Rank 1 to 3
for rank in range(1, 4):
    ax = axes[rank - 1]
    subset = df[df["Rank"] == rank]
    sns.countplot(data=subset, x="length_category", hue="Model", order=["long", "medium", "short"], ax=ax, palette=model_palette)
    ax.set_title(f"Rank {rank}")
    ax.set_xlabel("Length Category")
    ax.set_ylabel("Count")

# Improve layout
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
fig.savefig("retrieval_length_distribution.png", dpi=300, bbox_inches='tight')

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency


def perform_chi2_per_rank_length(df):
    """
    Performs Chi-Square test of independence between Model and Length Category
    for each rank in the DataFrame.

    Args:
        df (pd.DataFrame): DataFrame with 'Model', 'Rank', and 'length_category' columns
                           ('long', 'medium', 'short').

    Returns:
        dict: A dictionary where keys are ranks and values are the Chi-Square
              test results.
    """
    results = {}
    for rank in sorted(df['Rank'].unique()):
        rank_df = df[df['Rank'] == rank]
        contingency_table = pd.crosstab(rank_df['Model'], rank_df['length_category'])
        if contingency_table.shape[0] > 1 and contingency_table.shape[1] > 1:
            chi2, p, dof, expected = chi2_contingency(contingency_table)
            results[f"Rank {rank}"] = {'Chi2 Statistic': chi2,
                                       'P-value': p,
                                       'Degrees of Freedom': dof,
                                       'Expected Frequencies': expected}
        else:
            results[f"Rank {rank}"] = "Insufficient data for Chi-Square test"
    return results

def create_contingency_tables_length(df):
    """
    Creates contingency tables of Model vs. Length Category for each rank.

    Args:
        df (pd.DataFrame): DataFrame with 'Model', 'Rank', and 'length_category' columns.

    Returns:
        dict: A dictionary where keys are ranks and values are the
              corresponding contingency tables (pd.DataFrame).
    """
    contingency_tables = {}
    for rank in sorted(df['Rank'].unique()):
        rank_df = df[df['Rank'] == rank]
        contingency_tables[f"Rank {rank}"] = pd.crosstab(rank_df['Model'], rank_df['length_category'])
    return contingency_tables

# Perform the Chi-Square tests
chi2_length_results = perform_chi2_per_rank_length(df)

# Print the results
for rank, result in chi2_length_results.items():
    print(f"\n--- Rank {rank} (Length Bias) ---")
    if isinstance(result, str):
        print(result)
    else:
        print(f"Chi2 Statistic: {result['Chi2 Statistic']:.4f}")
        print(f"P-value: {result['P-value']:.4f}")
        print(f"Degrees of Freedom: {result['Degrees of Freedom']}")
        print("Expected Frequencies:\n", result['Expected Frequencies'])

# Create contingency tables for length bias
contingency_tables_length = create_contingency_tables_length(df)

# Print the contingency tables
print("\n--- Contingency Tables (Model vs. Length Category per Rank) ---")
for rank, table in contingency_tables_length.items():
    print(f"\nRank {rank}:\n{table}")